# Explorando a API do iFood

In [9]:
import requests
import json

## A url pode ser chamada de dois modos:

### Procurando vários restaurantes
- Usado pelo iFood para listar os restaurantes
- Url Base: https://marketplace.ifood.com.br/v1/merchants?
- Parâmetros (eles são divididos por &):
    - channel: quase sempre é obrigatória.
        - sempre usa-se channel=IFOOD (não achei outro caso, pelo menos)
    - latitude, longitude: use as duas ou não use nenhuma. 
        - exemplo: latitude=-23.19529&longitude=-45.90321
    - size, page: na hora de fazer requests sem especificar o restaurantid, você pode definir quantos restaurantes quer receber com a variável size, a page itera entre páginas de tamanho size. 
        - exemplo: size=100&page=2 para mostrar os resultados 101-200

### Procurando um restaurante específico, dado o id dele
- https://marketplace.ifood.com.br/v1/merchants/{id}/extra: retorna algumas informações sobre o restaurante do id enviado, como localização geográfica e descrição da loja ({id} é o id do restaurante)
    - Usado pelo iFood para criar a descrição do restaurante
- https://marketplace.ifood.com.br/v1/merchants/{id}/payment-methods: retorna algumas informações sobre o restaurante do id enviado, como localização geográfica e descrição da loja ({id} é o id do restaurante)
    - Usado pelo iFood para listar os métodos de pagamento disponíveis
- https://marketplace.ifood.com.br/v1/merchants/{id}?channel=IFOOD: retorna algumas informações referentes à entrega desse restaurante. zip_code, latitude e longitude também podem ser usados.
    - exemplo: https://marketplace.ifood.com.br/v2/merchants/1777ea72-8ae7-4b6c-a63e-9a4639d2d0b1?restaurantId=1777ea72-8ae7-4b6c-a63e-9a4639d2d0b1&latitude=-23.492296&longitude=-46.84311659999999&channel=IFOOD&size=10
    - outro exemplo: https://marketplace.ifood.com.br/v2/merchants/1777ea72-8ae7-4b6c-a63e-9a4639d2d0b1?restaurantId=1777ea72-8ae7-4b6c-a63e-9a4639d2d0b1&channel=IFOOD&zip_code=06440180

## a) Listando restaurantes numa área 
### Esse método foi usado para a criação do dataset dos restaurantes

In [19]:
# Para obter os restaurantes disponiveis em latitude=-23.19529, longitude=-45.90321:
response = requests.get("https://marketplace.ifood.com.br/v1/merchants?latitude=-23.19529&longitude=-45.90321&channel=IFOOD")
content = response.content
# Observando as dict_keys: "total" indica o numero de restaurantes na área, pode ser usado para nos ajudar a iterar pelas páginas, enquanto "merchants" contém uma lista informações sobre os restaurantes e "facets" dá um resumo das informações dos restaurantes.
json.loads(content).keys()

dict_keys(['page', 'total', 'size', 'merchants', 'facets'])

In [20]:
# o size = 0 esconde a lista merchants, que contém informações sobre os restaurantes, podemos usar isso para gastar menos dados na hora de obter algo das "facets" ou o "total"
response = requests.get("https://marketplace.ifood.com.br/v1/merchants?latitude=-23.19529&longitude=-45.90321&channel=IFOOD")
content = response.content
json.loads(content)['total']

2547

In [32]:
# "merchants" contém as informações utilizadas para gerar o dataset, o parametro size delimita o tamanho dele
response = requests.get("https://marketplace.ifood.com.br/v1/merchants?latitude=-23.19529&longitude=-45.90321&channel=IFOOD&size=2")
content = response.content
print("Tamanho da 'merchants':", len(json.loads(content)['merchants']))
# Exemplo de uma lista 'merchants':
json.loads(content)['merchants']

Tamanho da 'merchants': 2


[{'id': '74dde208-f99c-4e06-bb90-2bbbfaffeb7e',
  'name': 'Rei do Açai',
  'resources': [{'type': 'LOGO',
    'fileName': '74dde208-f99c-4e06-bb90-2bbbfaffeb7e/202006191726_EfW5_i.jpg'},
   {'type': 'HEADER',
    'fileName': '74dde208-f99c-4e06-bb90-2bbbfaffeb7e/202006191726_Ap7u_i.jpg'}],
  'slug': 'sao-jose-dos-campos-sp/rei-do-acai-bosque-dos-eucaliptos',
  'userRating': 4.2740302085876465,
  'priceRange': 'CHEAP',
  'mainCategory': {'code': 'AC1', 'name': 'Açaí'},
  'features': ['DELIVERY', 'SCHEDULING', 'IMMEDIATE_ORDER'],
  'tags': ['70OFF_OUTRAS_CIDADES_DFN',
   'ABR20_LANCHE',
   'ADDRESS_PREFORM_TYPE',
   'ALL_OFF_SET19',
   'ALMOCO_ATE_15',
   'ALMOCO_ATE_15_ELIGIBLE',
   'ALMOCO_ATE_15_OPTED_IN',
   'BF_CUPOM15OFF_NOV20',
   'CAFE_DA_MANHA',
   'CAFE_DA_MANHA_ELIGIBLE',
   'CAFE_DA_MANHA_OPTED_IN',
   'carrossel_madrugada_ago20',
   'CART::DELIVERY_FEE_DISH',
   'CART::DELIVERY_FEE_DISH_ELIGIBLE',
   'CART::DELIVERY_FEE_DISH_OPTED_IN',
   'CONTA_ESTRATEGICA',
   'CPGN_1811_B

## b) obtendo informações mais específicas de um restaurante
### Esse método pode ser usado para conseguir mais informações dos restaurantes presentes no dataset

In [49]:
# obtendo o id de um restaurante qualquer
response = requests.get("https://marketplace.ifood.com.br/v1/merchants?latitude=-23.19529&longitude=-45.90321&channel=IFOOD")
content = response.content
merchants = json.loads(content)['merchants']
# seleciona o primeiro restaurante da lista e obtém o id dele
restaurant_id = merchants[0]['id']
restaurant_id

'b279e970-562b-4180-8f0d-e62ae7cbdb5a'

In [38]:
# passando o id obtido anteriormente pra outra request
response = requests.get(f"https://marketplace.ifood.com.br/v1/merchants/{restaurant_id}/extra")
content = response.content
restaurant_info = json.loads(content)

In [45]:
# é possível obter o cnpj e o endereço do restaurante, por exemplo:
print(restaurant_info['address'])
print(restaurant_info['documents'])

{'district': 'Bosque dos Eucaliptos', 'city': 'São José dos Campos', 'state': 'SP', 'country': 'BR', 'latitude': -23.242613, 'longitude': -45.888443, 'timezone': 'Etc/GMT+3', 'zipCode': '12233400', 'streetName': 'Av ouro fino ', 'streetNumber': '818', 'streetCompl': 'Sala 03'}
{'CNPJ': {'type': 'CNPJ', 'value': '28267212000159'}}


## c) Outra API do iFood
Encontrei outra API, que pode ser acessada via:
-  https://wsloja.ifood.com.br/ifood-ws-v3/restaurants/{id}

E pode exibir o menu da loja se usarmos:
- https://wsloja.ifood.com.br/ifood-ws-v3/restaurants/{id}/menu

Mas recebi "Acesso não permitido". Debugando o site do iFood, parece ser necessário um token ou algo do tipo